Streaming DataFrames are largely the same as static DataFrames. We create them within Spark applications and then perform transformations on them to get our data into the correct format.
- Basically, all of the transformations that are available in the static Structured APIs apply to Streaming DataFrames. 
- However, one small difference is that Structured Streaming does not let you perform schema inference without explicitly enabling it. 
- You can enable schema inference for this by setting the configuration spark.sql.streaming.schemaInference to true.
-  Given that fact, we will read the schema from one file (that we know has a valid schema) and pass the dataSchema object from our static DataFrame to our streaming DataFrame.
-  As mentioned, you should avoid doing this in a production scenario where your data may (accidentally) change out from under you

In [0]:
static = spark.read.json("/FileStore/tables/activity_data")
dataSchema = static.schema


maxFilesPerTrigger
- allows you to control how quickly Spark will read all of the files in the folder
- By specifying this value lower, we’re artificially limiting the flow of the stream to one file per trigger. 
- This helps us demonstrate how Structured Streaming runs incrementally in our example, but probably isn’t something you’d use in production.

In [0]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
  .json("/FileStore/tables/activity_data")


Streaming DataFrame creation and execution is lazy
-  specify transformations on our streaming DataFrame before finally calling an action to start the stream. In this case, we’ll show one simple transformation—we will group and count data by the gt column, which is the activity being performed by the user at that point in time:

In [0]:
activityCounts = streaming.groupBy("gt").count()


specify our action to start the query.
-  specify an output destination, or output sink for our result of this query. 
- write to a memory sink which keeps an in-memory table of the results. 
- define how Spark will output that data. 
- use the complete output mode - rewrites all of the keys along with their counts after every trigger

In [0]:
activityQuery = activityCounts.writeStream.queryName("activity_counts")\
  .format("memory").outputMode("complete")\
  .start()


In [0]:
from time import sleep
for x in range(5):
    spark.sql("SELECT * FROM activity_counts").show()
    sleep(1)


+----------+------+
 gt| count|
+----------+------+
 stairsup|167255|
 sit|196927|
 stand|182165|
 walk|212095|
 bike|172762|
stairsdown|149819|
 null|167168|
+----------+------+

+----------+------+
 gt| count|
+----------+------+
 stairsup|177710|
 sit|209235|
 stand|193552|
 walk|225351|
 bike|183559|
stairsdown|159182|
 null|177614|
+----------+------+

+----------+------+
 gt| count|
+----------+------+
 stairsup|177710|
 sit|209235|
 stand|193552|
 walk|225351|
 bike|183559|
stairsdown|159182|
 null|177614|
+----------+------+

+----------+------+
 gt| count|
+----------+------+
 stairsup|188165|
 sit|221543|
 stand|204938|
 walk|238607|
 bike|194355|
stairsdown|168545|
 null|188061|
+----------+------+

+----------+------+
 gt| count|
+----------+------+
 stairsup|188165|
 sit|221543|
 stand|204938|
 walk|238607|
 bike|194355|
stairsdown|168545|
 null|188061|
+----------+------+

Selections and Filtering
-  All select and filter transformations are supported in Structured Streaming
-  as are all DataFrame functions and individual column manipulations
- We show a simple example using selections and filtering below. 
- In this case, because we are not updating any keys over time, we will use the Append output mode, so that new results are appended to the output table:

In [0]:
from pyspark.sql.functions import expr
simpleTransform = streaming.withColumn("stairs", expr("gt like '%stairs%'"))\
  .where("stairs")\
  .where("gt is not null")\
  .select("gt", "model", "arrival_time", "creation_time")\
  .writeStream\
  .queryName("simple_transform")\
  .format("memory")\
  .outputMode("append")\
  .start()


Aggregations
-  can specify arbitrary aggregations, as you saw in the Structured APIs
- can use a more exotic aggregation, like a cube, on the phone model and activity and the average x, y, z accelerations of our sensor
- (jump back to Chapter 7 in order to see potential aggregations that you can run on your stream)

In [0]:
deviceModelStats = streaming.cube("gt", "model").avg()\
  .drop("avg(Arrival_time)")\
  .drop("avg(Creation_Time)")\
  .drop("avg(Index)")\
  .writeStream.queryName("device_counts").format("memory")\
  .outputMode("complete")\
  .start()


In [0]:
display(spark.sql("SELECT * FROM device_counts"))

gt,model,avg(x),avg(y),avg(z)
null,nexus4,-0.007024379640278381,-5.438909838219232E-4,0.005195158657373937
null,nexus4,0.0011945763431121092,-0.0061174486583486214,-0.008591960711177812
null,null,0.0011945763431121092,-0.0061174486583486214,-0.008591960711177812
bike,nexus4,0.023853077485904806,-0.009183302863968828,-0.08251574232228004
stand,null,-4.055901934813167E-4,4.2886781289306545E-4,2.234958592930457E-4
sit,nexus4,-5.349253688532204E-4,3.42148544390284E-4,-1.266214101180325E-4
stand,nexus4,-4.055901934813167E-4,4.2886781289306545E-4,2.234958592930457E-4
stairsdown,null,0.026141112228300515,-0.03781945599450251,0.1259837445616258
stairsup,null,-0.02687827649018481,-0.008319002048281888,-0.09788031217647133
sit,null,-5.349253688532204E-4,3.42148544390284E-4,-1.266214101180325E-4


In [0]:
historicalAgg = static.groupBy("gt", "model").avg()
deviceModelStats = streaming.drop("Arrival_Time", "Creation_Time", "Index")\
  .cube("gt", "model").avg()\
  .join(historicalAgg, ["gt", "model"])\
  .writeStream.queryName("device_counts").format("memory")\
  .outputMode("complete")\
  .start()


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1752595641097406> in <module> 
 ----> 1 historicalAgg = static . groupBy ( "gt" , "model" ) . avg ( ) 
 2 deviceModelStats = streaming . drop ( "Arrival_Time" , "Creation_Time" , "Index" ) \ 
 3 . cube ( "gt" , "model" ) . avg ( ) \ 
 4 . join ( historicalAgg , [ "gt" , "model" ] ) \ 
 5 . writeStream . queryName ( "device_counts" ) . format ( "memory" ) \ 

 NameError : name 'static' is not defined

Reading from the Kafka Source
- To read, you first need to choose one of the following options: assign, subscribe, or subscribePattern
- Only one of these can be present as an option when you go to read from Kafka.
- Assign is a fine-grained way of specifying not just the topic but also the topic partitions from which you would like to read. 
- This is specified as a JSON string {"topicA": [0,1],"topicB":[2,4]}. subscribe and subscribePattern are ways of subscribing to one or more topics either by specifying a list of topics (in the former) or via a pattern (via the latter).
- Second, you will need to specify the kafka.bootstrap.servers that Kafka provides to connect to the service.

In [0]:
# Subscribe to 1 topic
df1 = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "host1:port1,host2:port2")\
  .option("subscribe", "topic1")\
  .load()
# Subscribe to multiple topics
df2 = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "host1:port1,host2:port2")\
  .option("subscribe", "topic1,topic2")\
  .load()
# Subscribe to a pattern
df3 = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "host1:port1,host2:port2")\
  .option("subscribePattern", "topic.*")\
  .load()


Writing to the Kafka Sink
- Writing to Kafka queries is largely the same as reading from them except for fewer parameters.
- You’ll still need to specify the Kafka bootstrap servers, but the only other option you will need to supply is either a column with the topic specification or supply that as an option.

In [0]:
df1.selectExpr("topic", "CAST(key AS STRING)", "CAST(value AS STRING)")\
  .writeStream\
  .format("kafka")\
  .option("kafka.bootstrap.servers", "host1:port1,host2:port2")\
  .option("checkpointLocation", "/to/HDFS-compatible/dir")\
  .start()
df1.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")\
  .writeStream\
  .format("kafka")\
  .option("kafka.bootstrap.servers", "host1:port1,host2:port2")\
  .option("checkpointLocation", "/to/HDFS-compatible/dir")\
  .option("topic", "topic1")\
  .start()


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
 File "/databricks/python/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
 File "<command-1752595641097408>", line 5, in <module>
 .option("checkpointLocation", "/to/HDFS-compatible/dir")\
 File "/databricks/spark/python/pyspark/sql/streaming.py", line 1225, in start
 File "/databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1305, in __call__
 answer, self.gateway_client, self.target_id, self.name)
 File "/databricks/spark/python/pyspark/sql/utils.py", line 127, in deco
 File "/databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 328, in get_return_value
 format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: An error occurred while calling o530.start.
: java.rmi.RemoteException: com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 15053); nested exception is: 
	com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 15053)
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIgnoreDraining(DbfsClient.scala:88)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close0(DbfsOutputStream.scala:100)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close(DbfsOutputStream.scala:79)
	at java.io.FilterOutputStream.close(FilterOutputStream.java:159)
	at org.apache.hadoop.fs.FSDataOutputStream$PositionCache.close(FSDataOutputStream.java:72)
	at org.apache.hadoop.fs.FSDataOutputStream.close(FSDataOutputStream.java:106)
	at org.apache.hadoop.fs.FSDataOutputStream$PositionCache.close(FSDataOutputStream.java:72)
	at org.apache.hadoop.fs.FSDataOutputStream.close(FSDataOutputStream.java:106)
	at sun.nio.cs.StreamEncoder.implClose(StreamEncoder.java:320)
	at sun.nio.cs.StreamEncoder.close(StreamEncoder.java:149)
	at java.io.OutputStreamWriter.close(OutputStreamWriter.java:233)
	at com.fasterxml.jackson.core.json.WriterBasedJsonGenerator.close(WriterBasedJsonGenerator.java:974)
	at com.fasterxml.jackson.databind.ObjectMapper._configAndWriteValue(ObjectMapper.java:4099)
	at com.fasterxml.jackson.databind.ObjectMapper.writeValue(ObjectMapper.java:3386)
	at org.json4s.jackson.Serialization$.write(Serialization.scala:27)
	at org.apache.spark.sql.execution.streaming.StreamMetadata$.write(StreamMetadata.scala:80)
	at org.apache.spark.sql.execution.streaming.StreamExecution.$anonfun$streamMetadata$1(StreamExecution.scala:182)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.streaming.StreamExecution.<init>(StreamExecution.scala:180)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution.<init>(MicroBatchExecution.scala:52)
	at org.apache.spark.sql.streaming.StreamingQueryManager.createQuery(StreamingQueryManager.scala:327)
	at org.apache.spark.sql.streaming.StreamingQueryManager.startQuery(StreamingQueryManager.scala:369)
	at org.apache.spark.sql.streaming.DataStreamWriter.start(DataStreamWriter.scala:390)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at su

---------------------------------------------------------------------------

Socket source
- The socket source allows you to send data to your Streams via TCP sockets. 
- To start one, specify a host and port to read data from. 
- Spark will open a new TCP connection to read from that address. 
- The socket source should not be used in production because the socket sits on the driver and does not provide end-to-end fault-tolerance guarantees. 
- Here is a short example of setting up this source to read from localhost:9999:

In [0]:
socketDF = spark.readStream.format("socket")\
  .option("host", "localhost").option("port", 9999).load()


Console sink 
- The console sink allows you to write out some of your streaming query to the console. 
- This is useful for debugging but is not fault-tolerant.
-  Writing out to the console is simple and only prints some rows of your streaming query to the console. This supports both append and complete output modes:

In [0]:
activityCounts.writeStream.trigger(processingTime='5 seconds')\
  .format("console").outputMode("complete").start()


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1752595641097410> in <module> 
 ----> 1 activityCounts . writeStream . trigger ( processingTime = '5 seconds' ) \ 
 2 . format ( "console" ) . outputMode ( "complete" ) . start ( ) 

 NameError : name 'activityCounts' is not defined

Once trigger
- You can also just run a streaming job once by setting that as the trigger. 
- This might seem like a weird case, but it’s actually extremely useful in both development and production. 
- During development, you can test your application on just one trigger’s worth of data at a time. 
- During production, the Once trigger can be used to run your job manually at a low rate (e.g., import new data into a summary table just occasionally). 
- Because Structured Streaming still fully tracks all the input files processed and the state of the computation, this is easier than writing your own custom logic to track this in a batch job, and saves a lot of resources over running a continuous job 24/7:

In [0]:
activityCounts.writeStream.trigger(once=True)\
  .format("console").outputMode("complete").start()


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
 File "/databricks/python/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
 File "<command-1752595641097411>", line 1, in <module>
 activityCounts.writeStream.trigger(once=True)\
NameError: name 'activityCounts' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
 File "/databricks/python/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
 File "/databricks/python/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1151, in get_records
 File "/databricks/python/lib/python3.7/site-packages/IPython/core/ultratb.py", line 319, in wrapped
 File "/databricks/python/lib/python3.7/site-packages/IPython/core/ultratb.py", line 353, in _fixed_getinnerframes
 File "/usr/lib/python3.7/inspect.py", line 1502, in getinnerframes
 File "/usr/lib/python3.7/inspect.py", line 1460, in getframeinfo
 File "/usr/lib/python3.7/inspect.py", line 696, in getsourcefile
 File "/usr/lib/python3.7/inspect.py", line 725, in getmodule
 File "/usr/lib/python3.7/inspect.py", line 709, in getabsfile
 File "/local_disk0/pythonVirtualEnvDirs/virtualEnv-977b0a77-1b3f-416b-ab28-ab489d3a1acc/lib/python3.7/posixpath.py", line 383, in abspath
FileNotFoundError: [Errno 2] No such file or directory

---------------------------------------------------------------------------